# Multi-Turn Conversations

Learn how to manage conversation history, context, and branching with lionpride's Branch system.

**What You'll Learn:**
- How Branch tracks conversation history
- Building multi-turn conversations with `communicate()`
- Using system messages for persona/behavior
- Forking conversations for "what if" scenarios
- Managing conversation context and persistence

**Prerequisites:** Basic understanding of Session and Message from earlier notebooks

In [ ]:
from lionpride import Message, Session
from lionpride.operations import communicate
from lionpride.services import iModel
from lionpride.session import SystemContent

## 1. Understanding Branch Structure

A **Branch** is a conversation thread that maintains message history:

```python
Branch
├── order: list[UUID]        # Message IDs in chronological order
├── system: UUID | None      # System message (first in order)
├── capabilities: set[str]   # Allowed structured outputs
├── resources: set[str]      # Allowed service resources
└── session_id: UUID         # Parent session
```

**Key Concepts:**
- Branch stores **UUID references**, actual messages live in `session.messages`
- Messages appear in chronological order via `branch.order`
- System message (if any) is always first
- Multiple branches can exist in one session (parallel conversations)

In [ ]:
# Create a session and inspect a branch
session = Session()
branch = session.create_branch(name="demo")

print(f"Branch ID: {branch.id}")
print(f"Branch name: {branch.name}")
print(f"Messages in branch: {len(branch.order)}")
print(f"System message: {branch.system}")

## 2. Basic Multi-Turn Chat

Each call to `communicate()` adds messages to the branch. The model sees the full conversation history.

In [ ]:
async def basic_multi_turn():
    """Simple multi-turn conversation with automatic context"""
    session = Session()
    model = iModel(provider="openai", model="gpt-4o-mini", temperature=0.7)
    session.services.register(model)

    branch = session.create_branch(name="python-help")

    # Turn 1: Initial question
    response1 = await communicate(
        session=session,
        branch=branch,
        parameters={
            "instruction": "What are Python lists?",
            "imodel": model.name,
        },
    )
    print("Turn 1:")
    print("User: What are Python lists?")
    print(f"Assistant: {response1[:200]}...\n")

    # Turn 2: Follow-up (has context from turn 1)
    response2 = await communicate(
        session=session,
        branch=branch,
        parameters={
            "instruction": "How do I add items to them?",
            "imodel": model.name,
        },
    )
    print("Turn 2:")
    print("User: How do I add items to them?")  # Notice the pronoun "them"
    print(f"Assistant: {response2[:200]}...\n")

    # Turn 3: Another follow-up
    response3 = await communicate(
        session=session,
        branch=branch,
        parameters={
            "instruction": "Show me an example with numbers",
            "imodel": model.name,
        },
    )
    print("Turn 3:")
    print("User: Show me an example with numbers")
    print(f"Assistant: {response3[:200]}...\n")

    # Inspect conversation
    print(f"Total messages in branch: {len(branch.order)}")
    print(f"Message IDs: {[str(mid)[:8] + '...' for mid in branch.order]}")

    return session, branch


# Run the conversation
session, branch = await basic_multi_turn()

**Notice**: In turn 2, we used "them" (pronoun reference) - the model understood from context!

## 3. Inspecting Message History

Access messages via `session.messages[msg_id]` using IDs from `branch.order`.

In [ ]:
# Inspect each message in the conversation
print("=== Conversation History ===")
print()

for i, msg_id in enumerate(branch.order, 1):
    message = session.messages[msg_id]
    role = message.role.value

    # Extract content based on message type
    if hasattr(message.content, "instruction"):
        content = message.content.instruction
    elif hasattr(message.content, "assistant_response"):
        content = message.content.assistant_response
    else:
        content = str(message.content)

    print(f"Message {i} [{role}]:")
    print(f"  {content[:100]}...")
    print(f"  Created: {message.created_at}")
    print(f"  ID: {message.id}")
    print()

## 4. System Messages and Personas

System messages define the model's behavior/personality. They're always the first message in a branch.

In [ ]:
async def persona_conversation():
    """Demonstrate system message influence on behavior"""
    session = Session()
    model = iModel(provider="openai", model="gpt-4o-mini", temperature=0.7)
    session.services.register(model)

    # Create system message for a Python tutor persona
    # Note: SystemContent uses 'system_message' field, not 'system'
    system_msg = Message(
        content=SystemContent(
            system_message="You are an enthusiastic Python tutor who loves using "
            "emojis and encourages students. Keep responses concise "
            "and always include a code example."
        )
    )

    branch = session.create_branch(name="tutoring", system=system_msg)

    # Ask a question
    response = await communicate(
        session=session,
        branch=branch,
        parameters={
            "instruction": "What are dictionaries?",
            "imodel": model.name,
        },
    )

    print("With enthusiastic tutor persona:")
    print(response)
    print()

    # Verify system message is first
    first_msg = session.messages[branch.order[0]]
    print(f"First message role: {first_msg.role.value}")
    print(f"System message ID matches: {branch.system == first_msg.id}")

    return session, branch


await persona_conversation()

**Notice**: The response style matches the system message's personality!

## 5. Multiple Branches in One Session

Different branches maintain independent conversation threads with separate contexts.

In [ ]:
async def multiple_branches_demo():
    """Parallel conversations in different branches"""
    session = Session()
    model = iModel(provider="openai", model="gpt-4o-mini", temperature=0.7)
    session.services.register(model)

    # Branch 1: Technical discussion
    tech_system = Message(
        content=SystemContent(
            system_message="You are a senior software engineer discussing architecture. "
            "Be precise and technical."
        )
    )
    tech_branch = session.create_branch(name="technical", system=tech_system)

    # Branch 2: Creative writing
    creative_system = Message(
        content=SystemContent(
            system_message="You are a creative writing coach. Be imaginative and "
            "encourage vivid storytelling."
        )
    )
    creative_branch = session.create_branch(name="creative", system=creative_system)

    # Parallel conversations
    tech_response = await communicate(
        session=session,
        branch=tech_branch,
        parameters={
            "instruction": "Explain microservices in 2 sentences",
            "imodel": model.name,
        },
    )

    creative_response = await communicate(
        session=session,
        branch=creative_branch,
        parameters={
            "instruction": "Write an opening line for a sci-fi story",
            "imodel": model.name,
        },
    )

    print("Technical Branch:")
    print(tech_response)
    print()

    print("Creative Branch:")
    print(creative_response)
    print()

    # Continue technical conversation (no creative context)
    tech_response2 = await communicate(
        session=session,
        branch=tech_branch,
        parameters={
            "instruction": "What are the main trade-offs?",
            "imodel": model.name,
        },
    )

    print("Technical Branch (Turn 2):")
    print(tech_response2)
    print()

    # Session statistics
    print("=== Session Statistics ===")
    print(f"Total branches: {len(session.branches)}")
    print(f"Total messages: {len(session.messages)}")
    print(f"Technical branch messages: {len(tech_branch.order)}")
    print(f"Creative branch messages: {len(creative_branch.order)}")

    return session


await multiple_branches_demo()

**Key Insight**: Branches maintain independent context. The technical branch doesn't know about the creative conversation!

## 6. Forking Conversations

Create a new branch starting from a specific point in an existing conversation ("what if" scenarios).

In [ ]:
async def forking_demo():
    """Fork a conversation to explore alternatives"""
    session = Session()
    model = iModel(provider="openai", model="gpt-4o-mini", temperature=0.7)
    session.services.register(model)

    # Start a conversation
    main_branch = session.create_branch(name="main")

    await communicate(
        session=session,
        branch=main_branch,
        parameters={
            "instruction": "I'm building a web app. Should I use REST or GraphQL?",
            "imodel": model.name,
        },
    )

    response = await communicate(
        session=session,
        branch=main_branch,
        parameters={
            "instruction": "Let's go with REST. What framework?",
            "imodel": model.name,
        },
    )

    print("Main branch (REST path):")
    print(response[:200], "...\n")

    # Fork at the decision point (before choosing REST)
    # Take first 2 messages (initial question + answer)
    fork_point = 2
    alternative_branch = session.create_branch(
        name="graphql-alternative", messages=main_branch.order[:fork_point]
    )

    # Explore alternative path
    alt_response = await communicate(
        session=session,
        branch=alternative_branch,
        parameters={
            "instruction": "Actually, let's explore GraphQL. What are the benefits?",
            "imodel": model.name,
        },
    )

    print("Alternative branch (GraphQL path):")
    print(alt_response[:200], "...\n")

    # Compare branch states
    print("=== Branch Comparison ===")
    print(f"Main branch messages: {len(main_branch.order)}")
    print(f"Alternative branch messages: {len(alternative_branch.order)}")
    print(
        f"Shared history (first {fork_point} messages): {main_branch.order[:fork_point] == alternative_branch.order[:fork_point]}"
    )

    return session


await forking_demo()

**Use Case**: A/B testing responses, exploring different solutions, or debugging conversation paths.

## 7. Explicit Context Management

Provide additional context alongside instructions using the `context` parameter.

In [ ]:
async def context_management():
    """Provide explicit context for more precise responses"""
    session = Session()
    model = iModel(provider="openai", model="gpt-4o-mini", temperature=0.7)
    session.services.register(model)

    branch = session.create_branch(name="code-review")

    # Turn 1: Provide code in context
    response1 = await communicate(
        session=session,
        branch=branch,
        parameters={
            "instruction": "Review this code for bugs",
            "context": {
                "code": """
def calculate_average(numbers):
    total = 0
    for num in numbers:
        total += num
    return total / len(numbers)
                """,
                "language": "Python",
                "focus": "edge cases",
            },
            "imodel": model.name,
        },
    )

    print("Code Review:")
    print(response1)
    print()

    # Turn 2: Follow-up with additional context
    response2 = await communicate(
        session=session,
        branch=branch,
        parameters={
            "instruction": "How would you fix it?",
            "context": {
                "requirement": "Handle empty lists gracefully",
                "style": "Use modern Python idioms",
            },
            "imodel": model.name,
        },
    )

    print("Fix Suggestion:")
    print(response2)

    return session, branch


await context_management()

**Best Practice**: Use `context` for structured data (code, configs, specs) separate from the instruction.

## 8. Conversation Persistence

Save and resume conversations across sessions using serialization.

In [ ]:
import json
from pathlib import Path


async def save_conversation():
    """Save conversation to disk"""
    session = Session()
    model = iModel(provider="openai", model="gpt-4o-mini", temperature=0.7)
    session.services.register(model)

    branch = session.create_branch(name="persistent")

    # Have a conversation
    await communicate(
        session=session,
        branch=branch,
        parameters={
            "instruction": "What are Python decorators?",
            "imodel": model.name,
        },
    )

    await communicate(
        session=session,
        branch=branch,
        parameters={
            "instruction": "Show me a practical example",
            "imodel": model.name,
        },
    )

    # Export conversation
    conversation_data = {
        "branch_id": str(branch.id),
        "branch_name": branch.name,
        "messages": [
            {"id": str(msg_id), "message": session.messages[msg_id].to_dict()}
            for msg_id in branch.order
        ],
    }

    # Save to file
    save_path = Path("/tmp/conversation.json")
    with open(save_path, "w") as f:
        json.dump(conversation_data, f, default=str, indent=2)

    print(f"✓ Conversation saved to {save_path}")
    print(f"  Messages: {len(branch.order)}")
    return save_path


save_path = await save_conversation()

In [ ]:
async def resume_conversation(save_path):
    """Resume conversation from disk"""
    # Create new session
    new_session = Session()
    new_model = iModel(provider="openai", model="gpt-4o-mini", temperature=0.7)
    new_session.services.register(new_model)

    # Load conversation
    with open(save_path) as f:
        loaded_data = json.load(f)

    # Reconstruct messages
    messages = []
    for msg_data in loaded_data["messages"]:
        msg = Message.from_dict(msg_data["message"])
        messages.append(msg)
        new_session.conversations.add_item(msg)  # Add to session

    # Recreate branch with message references
    new_branch = new_session.create_branch(
        name=loaded_data["branch_name"], messages=[msg.id for msg in messages]
    )

    print(f"✓ Resumed conversation: {new_branch.name}")
    print(f"  Messages loaded: {len(new_branch.order)}")
    print()

    # Continue conversation
    response = await communicate(
        session=new_session,
        branch=new_branch,
        parameters={
            "instruction": "Can you explain the example in more detail?",
            "imodel": new_model.name,
        },
    )

    print("Continued conversation:")
    print(response[:300], "...")
    print()
    print(f"Total messages now: {len(new_branch.order)}")

    return new_session, new_branch


await resume_conversation(save_path)

**Use Case**: Long-running conversations, session recovery, conversation analysis.

## 9. Managing Context Windows

Limit message history to stay within model context windows.

In [ ]:
def get_recent_messages(session, branch, n=5):
    """Get last N messages from a branch"""
    recent_ids = branch.order[-n:]
    return [session.messages[msg_id] for msg_id in recent_ids]


# Example: Get last 3 messages
session = Session()
branch = session.create_branch(name="test")

# Simulate multiple messages
for i in range(10):
    msg = Message(content={"instruction": f"Message {i}"})
    session.conversations.add_item(msg)
    branch.order.append(msg.id)

recent = get_recent_messages(session, branch, n=3)
print(f"Total messages: {len(branch.order)}")
print(f"Recent messages: {len(recent)}")
print(f"Recent IDs: {[str(m.id)[:8] + '...' for m in recent]}")

**Pattern**: Create a sliding window of recent messages for very long conversations.

## Summary Checklist

**What You Learned:**
- ✅ Branch tracks message history via `order: list[UUID]`
- ✅ `communicate()` automatically builds multi-turn context
- ✅ System messages define model behavior (always first in branch)
- ✅ Multiple branches enable parallel conversations
- ✅ Forking creates alternative conversation paths
- ✅ `context` parameter provides structured data
- ✅ Serialization enables conversation persistence
- ✅ Sliding windows manage long conversations

**Common Pitfalls:**
- ❌ Don't modify `branch.order` directly - use `communicate()`
- ❌ Don't forget to add messages to `session.conversations` first
- ❌ Don't assume branches share context - they're independent
- ❌ Don't forget system message is always `branch.order[0]`

**Next Steps:**
- See notebook 06: Tool calling with conversation context
- See notebook 07: Streaming multi-turn responses
- See notebook 08: Multi-agent conversations

**Architecture Insight:**
```python
Session
├── conversations: Flow[Message, Branch]
│   ├── messages (Pile[Message])      # All messages
│   └── branches (Pile[Branch])       # All threads
└── services: ServiceRegistry
    └── models, tools
```

Branch = Progression with conversation-specific methods. Messages = Nodes with chat-specific content types.